In [1]:
# Have to use my_gbd_sunset environment
import os
# Shared Functions
from db_queries import get_location_metadata, get_age_metadata, get_population, get_cause_metadata, get_outputs
from get_draws.api import get_draws

In [2]:
# Directories
DATA_DIR = '/mnt/team/idd/pub/manuscripts/GBD2023_DENV/data'

In [3]:
# Run numbers
release_id = 16
como_v = 1762
codcorrect_v = 497
dalynator_v = 98
burdenator_v = 395
compare_v = 8305

data_date = "2025_09_30"
dengue_id = 357
dengue_sequela_ids = [157, 158, 159]

# make data_date subdirectory within DATA_DIR
data_date_dir = f'{DATA_DIR}/{data_date}'
os.makedirs(data_date_dir, exist_ok=True)

In [ ]:
modeling_hierarchy_2023 = get_location_metadata(location_set_id = 35, release_id = release_id)
modeling_hierarchy_2023 = modeling_hierarchy_2023[['location_set_id', 'location_id', 'parent_id', 'path_to_top_parent', 'level', 'most_detailed', 'sort_order', 
                         'location_name', 'location_name_short', 'location_type', 'map_id', 'super_region_id', 'super_region_name',
                         'region_id', 'region_name', 'ihme_loc_id', 'local_id', 'lancet_label']]

SDI_Q = get_location_metadata(location_set_id=40, release_id=release_id)
SDI_names = ["Low SDI", "Low-middle SDI", "Middle SDI", "High-middle SDI", "High SDI"]
SDI_Q['SDI_Q'] = SDI_Q['region_name']

modeling_hierarchy_2023 = modeling_hierarchy_2023.merge(SDI_Q[['location_id', 'SDI_Q']], how='left', on='location_id')
modeling_hierarchy_2023.to_parquet(f"{data_date_dir}/modeling_hierarchy_2023.parquet", index=False)


loc_global = modeling_hierarchy_2023[modeling_hierarchy_2023['level'] == 0]
loc_super_region = modeling_hierarchy_2023[modeling_hierarchy_2023['level'] == 1]
loc_region = modeling_hierarchy_2023[modeling_hierarchy_2023['level'] == 2]
loc_country = modeling_hierarchy_2023[modeling_hierarchy_2023['level'] == 3]
loc_sub_nat = modeling_hierarchy_2023[modeling_hierarchy_2023['level'] == 4]
loc_most_detailed = modeling_hierarchy_2023[modeling_hierarchy_2023['most_detailed'] == 1]

age_metadata = get_age_metadata(release_id = release_id)
age_metadata.to_parquet(f"{data_date_dir}/age_metadata.parquet", index=False)
age_group_ids = [1, 22, 27] + age_metadata['age_group_id'].tolist()
sex_ids = [1, 2, 3]
year_ids = list(range(1990, 2024))

In [ ]:
# Get population
all_population = get_population(
    age_group_id=age_group_ids,
    release_id=release_id,
    year_id=year_ids,
    location_id=list(loc_global['location_id']) +
                list(loc_super_region['location_id']) +
                list(loc_region['location_id']) +
                list(loc_country['location_id']),
    sex_id=sex_ids
)
all_population = all_population[["age_group_id", "location_id", "year_id", "sex_id", "population"]]
all_population.to_parquet(f"{data_date_dir}/population.parquet", index=False)

In [6]:
cause_meta = get_cause_metadata(cause_set_id=3, release_id=release_id)
level_3A_cause = cause_meta[(cause_meta['level'] == 3) & (cause_meta['cause_outline'] == 'A')]
all_CD_results = get_outputs(
    "cause",
    cause_id=level_3A_cause['cause_id'].tolist(),
    measure_id=2,
    year_id=[1990, 2000, 2023],
    location_id=loc_global['location_id'].tolist() + loc_super_region['location_id'].tolist() + loc_region['location_id'].tolist(),
    age_group_id=[22, 27],
    release_id=release_id,
    metric_id=[1, 3],
    sex_id=3,
    compare_version_id=compare_v
)
all_CD_results.to_parquet(f"{data_date_dir}/all_CD_results.parquet", index=False)                                      

In [7]:
dengue_sequelae_results = get_outputs(
    "sequela",
    sequela_id=dengue_sequela_ids,
    year_id=[1990, 2000, 2023],
    location_id=modeling_hierarchy_2023['location_id'].tolist(),
    age_group_id=age_group_ids,
    release_id=release_id,
    compare_version_id=compare_v,
    metric_id=[1, 3],
    sex_id = 'all'
)
dengue_sequelae_results.to_parquet(f"{data_date_dir}/dengue_sequelae_results.parquet", index=False)

In [8]:
aa_results = get_outputs(
    "cause",
    cause_id=dengue_id,
    measure_id='all',
    year_id=year_ids,
    location_id=modeling_hierarchy_2023['location_id'].tolist(),
    age_group_id=[22, 27],
    release_id=release_id,
    metric_id=[1, 3],
    sex_id='all',
    compare_version_id=compare_v
)
aa_results.to_parquet(f"{data_date_dir}/aa_results.parquet", index=False)

In [9]:
as_results = get_outputs(
    "cause",
    cause_id=dengue_id,
    measure_id='all',
    year_id=2023,
    location_id=modeling_hierarchy_2023['location_id'].tolist(),
    age_group_id='all',
    release_id=release_id,
    metric_id=[1, 3],
    sex_id='all',
    compare_version_id=compare_v
)
as_results.to_parquet(f"{data_date_dir}/as_results.parquet", index=False)

In [10]:
denv_2023_global_daly_draws = get_draws(gbd_id_type = "cause_id", 
                                          gbd_id = dengue_id, 
                                          location_id= 1, 
                                          year_id=2023, 
                                          source="dalynator", 
                                          measure_id = 2,
                                          metric_id = [1,2],
                                          sex_id = 3,
                                          release_id = release_id, 
                                          version_id = dalynator_v)
denv_2023_global_daly_draws.to_parquet(f"{data_date_dir}/denv_2023_global_daly_draws.parquet", index=False)

In [11]:
denv_2023_global_yll_draws = get_draws(gbd_id_type = "cause_id", 
                                         gbd_id = dengue_id, 
                                         location_id= 1, 
                                         year_id = 2023, 
                                         source = "codcorrect", 
                                         measure_id = 4,
                                         metric_id = [1,3],
                                         sex_id = 3,
                                         release_id = release_id, 
                                         version_id = codcorrect_v)
denv_2023_global_yll_draws.to_parquet(f"{data_date_dir}/denv_2023_global_yll_draws.parquet", index=False)

In [12]:
denv_2023_global_yld_draws = get_draws(gbd_id_type = "cause_id", 
                                         gbd_id = dengue_id, 
                                         location_id = 1, 
                                         year_id = 2023, 
                                         source = "como", 
                                         measure_id = 3,
                                         metric_id = [1,3],
                                         sex_id = 3,
                                         release_id = release_id, 
                                         version_id = como_v)
denv_2023_global_yld_draws.to_parquet(f"{data_date_dir}/denv_2023_global_yld_draws.parquet", index=False)

In [13]:
#prev =5 , inc =6 , deaths =1 , dalys =2 , ylds = 3, ylls = 4
denv_2023_global_deaths_draws = get_draws(gbd_id_type = "cause_id", 
                                        gbd_id = dengue_id, 
                                        location_id= 1, 
                                        year_id = 2023, 
                                        source = "codcorrect", 
                                        measure_id = 1,
                                        metric_id = [1,3],
                                        sex_id = 3,
                                        release_id = release_id, 
                                        version_id = codcorrect_v)

denv_2023_global_deaths_draws.to_parquet(f"{data_date_dir}/denv_2023_global_deaths_draws.parquet", index=False)

In [14]:
denv_2023_global_incidence_draws = get_draws(gbd_id_type = "cause_id", 
                                            gbd_id = dengue_id, 
                                            location_id = 1, 
                                            year_id = 2023, 
                                            source = "como", 
                                            measure_id = 6,
                                            metric_id = [1,3],
                                            sex_id = 3,
                                            release_id = release_id, 
                                            version_id = como_v)
denv_2023_global_incidence_draws.to_parquet(f"{data_date_dir}/denv_2023_global_incidence_draws.parquet", index=False)

In [15]:
all_CD_results_draws = get_draws(gbd_id_type = "cause_id", 
                                   gbd_id = level_3A_cause['cause_id'].tolist(), 
                                   location_id=loc_global['location_id'].tolist() + loc_super_region['location_id'].tolist(), 
                                   year_id=[1990, 2000, 2023], 
                                   source="dalynator",
                                   age_group_id = [22, 27],
                                   measure_id = 2,
                                   sex_id = 3,
                                   metric_id = [1,3], 
                                   release_id = release_id, 
                                   version_id = dalynator_v)
all_CD_results_draws.to_parquet(f"{data_date_dir}/all_CD_results_draws.parquet", index=False)

In [ ]:
denv_A0_DALY_draws = get_draws(gbd_id_type = "cause_id", 
                                 gbd_id = dengue_id, 
                                 location_id = loc_country['location_id'].tolist(),
                                 year_id=[1990, 2000, 2023], 
                                 source="dalynator",
                                 age_group_id = [22,27],
                                 measure_id = 2,
                                 sex_id = 3,
                                 metric_id = [1,3], 
                                 release_id = release_id, 
                                 version_id = dalynator_v)
denv_A0_DALY_draws.to_parquet(f"{data_date_dir}/denv_A0_DALY_draws.parquet", index=False)

In [ ]:
# denv sequale
sequela_draws = get_draws(gbd_id_type = "sequela_id", 
                        gbd_id = [157, 158, 159], 
                        location_id = 1,
                        year_id = 2023, 
                        source = "como",
                        measure_id = 3,
                        sex_id = 3,
                        metric_id = [1,3], 
                        release_id = release_id, 
                        version_id = como_v)
sequela_draws.to_parquet(f"{data_date_dir}/sequela_draws.parquet", index=False)